# PUTTING 3rd LEVEL KNOWN DATA INTO DATABASE:

Uses **matched_2ndlevel.csv** and checks for match on **[DomainResearch].dbo.orbisData**, and then inserts the data into **[DomainResearch].dbo.thirdLevelOrbis** table

In [ ]:
import io
import subprocess
import pyodbc
import tldextract as tlde
import csv

cnxn = pyodbc.connect('DRIVER={ODBC Driver 17 for SQL Server};SERVER=localhost;DATABASE=DomainResearch;UID=dr;PWD=dr2020')
sql = "select * from orbisData"
cursor = cnxn.cursor()
cursor.execute(sql)

#NOTE:  I HAVE THIS DISABLED IN THE JUPYTER NOTEBOOK BECAUSE IT CAN TAKE A LONG TIME TO RUN....
#       It works, it just is probably better to run in a terminal to keep an eye on it.
#
#  comment out the following line to run it for realz
assert(False)

orbisData = {}

while True:
    row = cursor.fetchone()
    if not row:
        break
    orbisData[row[4]] = row[0] #map the domain name to the id in the table
    #print(row[4], orbisData[row[4]])

with open('matched_2ndlevel.csv') as csvfile:
    readCSV = csv.reader(csvfile, delimiter=',')

    for row in readCSV:
        found = orbisData.get(row[0] + "." + row[1])
        if found and (len(row[2]) > 0) and ("." not in row[2]):
            #print(orbisData[row[0] +:w "." + row[1]])
            #put it into the database
            insertsql = "insert into thirdLevelOrbis (thirdlevel, orbisDataId) values ('" + row[2] + "', " + str(found) + ")"
            cnxn.cursor().execute(insertsql)
            cnxn.cursor().commit()
                      
#closing conn1Gection  
cnxn.close()
